In [1]:
function newtmin_test_wolfe(f,g,new_f,new_g,alpha,p)
    c1 = 1e-4                #Armijo condition constant
    c2 = 0.9                 #curvature condition constant
    
    armijo = new_f - f <= c1*alpha*(g'*p)[1,1]           #satisfies Armijo condition?
    curvature = abs(new_g'*p)[1,1] > -c2*(g'*p)[1,1]     #satisfies curvature condition?
    valley = (new_g'*p)[1,1] < 0                         #avoid hills?
    return (armijo, curvature, valley)
end

function newtmin( obj, x0; maxIts=100, optTol=1e-6, BFGS=false)
# Minimize a function f using Newton’s method.
# obj: a function that evaluates the objective value, gradient, and Hessian at a point x, i.e.,
#    (f, g, H) = obj(x)
# x0: starting point.
# maxIts (optional): maximum number of iterations.
# optTol (optional): optimality tolerance based on ||grad(x)|| <= optTol*||grad(x0)||
# BFGS (optional): set as true to use BFGS algorithm ( = false by default)

    verbose = false
    
    its = 0;
    x = x0;
    
    if(BFGS)
        (f,g,_) = obj(x)
        n = size(g)[1]
        H_inv = eye(n)/norm(g)        #approximate Hessian as scaled identity
    else
        (f,g,H) = obj(x)     #evaluate gradient, et al
    end
      
    ngx0 = norm(g)           #need to keep this value for stopping condition
    
    if(ngx0 > optTol^2)      #trap for low grad(x0)
        
        while(its < maxIts && norm(g) > optTol*(ngx0))
            
            Hessian_modded = false
            
            if(BFGS)
                p = -H_inv * g
            else
                p = -H \ g           #compute descent direction
            end
            
            alpha = 1
            alpha_min = 0
            alpha_max = 1
            
            new_x = x + alpha*p  #trial step
            
            
            (new_f,new_g,_) = obj(new_x)
            
            (armijo, curvature, valley) = newtmin_test_wolfe(f,g,new_f,new_g,alpha,p)
                
            while(!armijo || !valley)
                
                if(!Hessian_modded && !BFGS)
                    #force the Hessian to be positive definite
                    
                    if(verbose) print("H"); end
                    
                    Hessian_modded = true
                
                    (V, S) = eig(H)                    #decompose H
                
                    if(minimum(V) < 0)                 #if not positive definite
                    
                        #V[V .<= 0] = maximum(V)        #do not explore areas of negative curvature
                        
                        V = V + abs(1.01*minimum(V))
                    
                        V_inv = diagm(1 ./ V)
                    
                        p = -S*V_inv*S'*g              #recalculate p    
                        
                    end
                
                else    #Hessian was already fixed
                
                    if(!armijo)
                        alpha_max = alpha                 #not enough decrease; try a smaller step
                        if(verbose) print("A"); end
                    elseif(!curvature)
                        if(verbose) print("C"); end
                        break                             #decrease is too slow to worry about hills/valleys
                    else
                        if(verbose) print("V"); end
                        alpha_min = alpha                 #going to land on a hill, try to speed over it
                    end
                                
                    alpha = 0.5*(alpha_max + alpha_min)   #try an intermediate value
                end
                
                new_x = x + alpha*p                   #trial step
            
                (new_f,new_g,_) = obj(new_x)      #evaluate new gradient, et al
                
                
                (armijo, curvature, valley) = newtmin_test_wolfe(f,g,new_f,new_g,alpha,p)
            
                if (alpha_max - alpha_min < 0.01)       #interval is too narrow
                    if(verbose) print("B"); end
                    break        #break out of loop; just take a step and hope for the best
                end
            
            end        #terminate linesearch
            
            
            #calculate Hessian
            if (BFGS)
                y = new_g - g
                s = alpha*p
                ro = 1 / (y'*s)[1,1]
                H_inv = (eye(n) - ro*s*y')*H_inv*(eye(n) - ro*y*s') + ro*s*s'    
            else
                (_,_,H) = obj(new_x)
            end
            
            x = new_x           #commit to the step
            f = new_f
            g = new_g
            
            Hessian_modded = false
            
            its = its + 1
            
        end
    
    else
        println("grad(x0) is already < ", optTol^2)
    end
    
    return (x, its)
    
end

newtmin (generic function with 1 method)

In [2]:
#test with a simple example function
function fun(x)
    f = x[1]^6 + x[2]^4 - x[1]*x[2]^3
    g = [6x[1]^5 - x[2]^3; 4x[2]^3 - 3x[1]*x[2]^2]
    H = [30x[1]^4     3x[2]^2              ;
         3x[2]^2      12x[2]^2 - 6x[1]*x[2]]
    return (f,g,H)
end

x0 = [-1;1]

(f0,g0,H0) = fun(x0)

(x, its) = newtmin( fun, x0;  BFGS = true)

(f,g,H) = fun(x)

println("Ratio: ", norm(g)/norm(g0), " in ", its, " iterations")    #should be below optTol

sleep(0.1)    #to keep output from bleeding into the next cell

Ratio: 1.2082776971836815e-7 in 23 iterations


In [3]:
using Toms566

In [4]:
#test the algorithm on Toms566

max_its = 2000
tol = 1e-10

for number = 1:18
       
    p = Problem(number)

    (x, its) = newtmin(xk -> (p.obj(xk), p.grd(xk), p.hes(xk)), p.x0; maxIts = max_its, optTol = tol, BFGS = true)
                                    #newtmin(anonymous function, p.x0)
    
    g = p.grd(x)
    g0 = p.grd(p.x0)
    f = p.obj(x)
    f0 = p.obj(p.x0)
    
    if(norm(g)/norm(g0) <= tol || norm(g0) <= tol^2)
        println("Problem #", number, " solved in ", its, " iterations.  f* = ", f)
        #println("f0: ", f0, ", f: ", f)
    else
        println("FAIL on Problem #", number)#": normg0:", norm(g0), ", normg:", norm(g))
        #println("f0: ", f0, ", f: ", f)
    end
end

sleep(0.1)    #to keep output from bleeding into the next cell

Problem #1 solved in 30 iterations.  f* = 8.108845013103398e-17
Problem #2 solved in 43 iterations.  f* = 0.005655649925499936
Problem #3 solved in 11 iterations.  f* = 1.1279327696188855e-8
Problem #4 solved in 168 iterations.  f* = 1.446993403066809e-20
Problem #5 solved in 78 iterations.  f* = 3.622339039432224e-21
Problem #6 solved in 27 iterations.  f* = 8.923832630175931e-5
Problem #7 solved in 105 iterations.  f* = 1.3997601380989349e-6
Problem #8 solved in 26 iterations.  f* = 0.0005415748924911815
Problem #9 solved in 609 iterations.  f* = 88.03185764468085
Problem #10 solved in 30 iterations.  f* = 1.792730780542459e-26
Problem #11 solved in 41 iterations.  f* = 85822.2016263563
Problem #12 solved in 54 iterations.  f* = 3.823631870661858e-20
Problem #13 solved in 72 iterations.  f* = 2.139604678598898e-6
Problem #14 solved in 42 iterations.  f* = 1.6645013453885573e-21
Problem #15 solved in 78 iterations.  f* = 2.2111222450722513e-15
Problem #16 solved in 17 iterations.  f* 

In [5]:
#test on logistic regression set
(raw_data,headings) = readdlm("binary.csv",','; header = true)
y = raw_data[:, 1]
U = raw_data[:, 2:3]          #only 2:3 since instructions state only to use two features
(m,n) = size(U)
U = [U ones(m,1)]             #append 1s as the last feature (for regularization

function evalLogReg(U,y, a)
    (row,) = size(a)
    x = exp(U*a)              #this is the "inner" function of the sigmoid
    sig = x ./ (1 + x)        #value of the sigmoid for this iteration
    
    f = sum(-y .* U*a + log(1+x))[1,1]
    g = U'*(sig - y)          #gradient of L
    
    gsig = diagm(sig.*(1-sig))    #sigmoid derivative without chain rule term, as a diagonal matrix
    H = U'*gsig*U
    
    return (f,g,H)
end

evalLogReg (generic function with 1 method)

In [6]:
max_its = 100
tol = 1e-6
x0 = rand(3)/800
(x, its) = newtmin(xk -> evalLogReg(U,y,xk), x0; maxIts = max_its, optTol = tol, BFGS = false)

(f0,g0,H0) = evalLogReg(U,y,x0)
(f,g,H) = evalLogReg(U,y,x)

println("Ratio: ", norm(g)/norm(g0), " in ", its, " iterations")    #should be below optTol


#this took approximately 9,000 iterations using LLS (see HW2-alt.jl)
#Now it only takes 3-4 iterations with this implementation of Newton's method.

Ratio: 3.6033793994034693e-11 in 4 iterations
